In [6]:
import pandas as pd
import gzip
import spacy

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [2]:
import os

directory = os.fsencode("./")

categories = []

for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".gz"):
        print(filename)
        questions = getDF(filename)
        questions = questions[["question", "questionType"]]
        categories.append(questions)

qa_Video_Games.json.gz
qa_Patio_Lawn_and_Garden.json.gz
qa_Home_and_Kitchen.json.gz
qa_Electronics.json.gz
qa_Grocery_and_Gourmet_Food.json.gz
qa_Automotive.json.gz
qa_Arts_Crafts_and_Sewing.json.gz
qa_Tools_and_Home_Improvement.json.gz
qa_Sports_and_Outdoors.json.gz
qa_Office_Products.json.gz
qa_Cell_Phones_and_Accessories.json.gz
qa_Baby.json.gz
qa_Clothing_Shoes_and_Jewelry.json.gz
qa_Appliances.json.gz
qa_Beauty.json.gz
qa_Pet_Supplies.json.gz
qa_Musical_Instruments.json.gz
qa_Industrial_and_Scientific.json.gz
qa_Health_and_Personal_Care.json.gz
qa_Software.json.gz
qa_Toys_and_Games.json.gz


In [9]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [10]:
interview_questions = [
    {"question": "Have you done this before?", "questionType":"yes/no"},
    {"question": "How many times have you done this", "questionType":"yes/no"},
    {"question": "Can you tell me about youself", "questionType":"open-ended"},
    {"question": "Can I ask how that makes you feel?", "questionType": "open-ended"},
    {"question": "Can you tell me how that makes you feel?", "questionType": "open-ended"},
    {"question": "Can I ask why you feel like that?", "questionType": "open-ended"},
    {"question": "Can you tell me why you feel like that?", "questionType": "open-ended"},
    {"question": "Can you give me an example of that?", "questionType": "open-ended"},
    {"question": "Can you tell me about a significant experience you had?", "questionType": "open-ended"},
    {"question": "Can I ask what makes you feel like that?", "questionType": "open-ended"},
    {"question": "Can you describe what your test was like?", "questionType": "open-ended"},
    {"question": "Can you describe why you like that?", "questionType": "open-ended"},
    {"question": "Can you describe how you did that?", "questionType": "open-ended"},
    {"question": "Would you reccommend it to a friend?", "questionType": "yes/no"},
    {"question": "Could you reccommend it to a friend?", "questionType": "yes/no"},
    {"question": "Should I reccommend it to a friend?", "questionType": "yes/no"},
    {"question": "Could you describe how you did that?", "questionType": "open-ended"},
]

pd.DataFrame.from_dict(interview_questions)

,question,questionType
0,Have you done this before?,yes/no
1,How many times have you done this,yes/no
2,Can you tell me about youself,open-ended
3,Can I ask how that makes you feel?,open-ended
4,Can you tell me how that makes you feel?,open-ended
5,Can I ask why you feel like that?,open-ended
6,Can you tell me why you feel like that?,open-ended
7,Can you give me an example of that?,open-ended
8,Can you tell me about a significant experience...,open-ended
9,Can I ask what makes you feel like that?,open-ended


In [11]:
categories.append(pd.DataFrame.from_dict(interview_questions))

In [12]:
keywords = {
    "open-ended": {

    },
    "yes/no": {

    }
}

question_words = {
    
}

average_word_lengths = {
    "open-ended": {

    },
    "yes/no": {

    }
}

word_lengths = {
    "open-ended": [],
    "yes/no": []
}

In [13]:
for i, questions in enumerate(categories):
    print("Category: "+str(i+1)+"/"+str(len(categories)))
    for index, question in questions.sample(n=(25000 if questions.shape[0] > 25000 else questions.shape[0]) , random_state=1).iterrows():
        doc = nlp(question["question"])
#         token = doc[0]
#         if token.pos_ in ["AUX", "ADV"]:
#             word_lengths[question["questionType"]].append(len(doc))
#             keyword = token.text.lower()
#             if keyword in keywords[question["questionType"]].keys():
#                 keywords[question["questionType"]][keyword] += 1
#                 average_word_lengths[question["questionType"]][keyword].append(len(doc))
#             else:
#                 keywords[question["questionType"]][keyword] = 1
#                 average_word_lengths[question["questionType"]][keyword] = [len(doc)]

#             if keyword in question_words.keys():
#                 question_words[keyword] += 1
#             else:
#                 question_words[keyword] = 1
        
        words = []
        
        for i in range(len(doc)):
            if doc[i].pos_ in ["AUX", "ADV"] or doc[i].text in ["what", "who", "can", "which", "would", "could", "should"] :
                words.append(doc[i].text.lower())
            else:
                words.append("*")
                
#         for i in range(1, len(words)):
        words += ["*" for i in range(6)]
        keyword = " ".join(words[:6])
        if any(c.isalpha() for c in keyword):
            word_lengths[question["questionType"]].append(len(doc))
            if keyword in keywords[question["questionType"]].keys():
                keywords[question["questionType"]][keyword] += 1
                average_word_lengths[question["questionType"]][keyword].append(len(doc))
            else:
                keywords[question["questionType"]][keyword] = 1
                average_word_lengths[question["questionType"]][keyword] = [len(doc)]

            if keyword in question_words.keys():
                question_words[keyword] += 1
            else:
                question_words[keyword] = 1

Category: 1/1


In [14]:
open_words = pd.DataFrame.from_dict(keywords["open-ended"], orient="index", columns=["count"]).sort_values(by="count", ascending=False)
closed_words = pd.DataFrame.from_dict(keywords["yes/no"], orient="index", columns=["count"]).sort_values(by="count", ascending=False)

open_words["word"] = open_words.index
closed_words["word"] = closed_words.index

open_words['freq'] = open_words['count'].apply(lambda count: count/len(word_lengths["open-ended"]))
closed_words['freq'] = closed_words['count'].apply(lambda count: count/len(word_lengths["yes/no"]))

avr_length_open = sum(word_lengths["open-ended"]) / len(word_lengths["open-ended"])
avr_length_closed = sum(word_lengths["yes/no"]) / len(word_lengths["yes/no"])

In [15]:
open_words.head(40)

,count,word,freq
* * * what * *,2,* * * what * *,0.222222
* * * how * did,2,* * * how * did,0.222222
* * * why * *,2,* * * why * *,0.222222
* * * how * *,1,* * * how * *,0.111111
* * * * why *,1,* * * * why *,0.111111
* * * * how *,1,* * * * how *,0.111111


In [16]:
closed_words.head(50)

,count,word,freq
how * * have * *,1,how * * have * *,0.5
have * * * * *,1,have * * * * *,0.5


In [19]:
import json
def dict_predict(question):
    with open('question_data.json') as json_file:
        data = json.load(json_file)
    words = question.lower().split(" ")
    for structure in data.keys():
        elements = structure.split(" ")
        match = True
        for i, element in enumerate(elements):
            if i < len(words):
                if element != "*" and element != words[i]:
                    match = False
            
        if match:
            return data[structure]
    return 0.5

In [20]:
dict_predict("Does this come with a remote for each or does the same remote work both?")

0.21679373476479435

In [21]:
def predict(question):
    word = str(question[0]).lower()
    length = len(question)
    if word in open_words.index.values and word in closed_words.index.values:
        open_freq = open_words.loc[word]["freq"]
        closed_freq = closed_words.loc[word]["freq"]
        
        open_count = open_words.loc[word]["count"]
        closed_count = closed_words.loc[word]["count"]    
        
        prediction = open_count / (open_count + closed_count)
        
    elif word in open_words.index.values:
        prediction = 1
    elif word in closed_words.index.values:
        prediction = 0
    else:
        prediction = 0.5
    
    if prediction > 0.5:
        return "open-ended", prediction
    elif prediction < 0.5:
        return "yes/no", prediction
    else:
        return "unknown", prediction

In [22]:
question_trigger_words = pd.DataFrame.from_dict(question_words, orient="index", columns=["count"]).sort_values(by="count", ascending=False)

question_trigger_words["word"] = question_trigger_words.index
question_trigger_words["type"] = question_trigger_words["word"].apply(lambda word: predict([word]))
question_trigger_words

,count,word,type
* * * what * *,2,* * * what * *,"(open-ended, 1)"
* * * how * did,2,* * * how * did,"(open-ended, 1)"
* * * why * *,2,* * * why * *,"(open-ended, 1)"
* * * how * *,1,* * * how * *,"(open-ended, 1)"
* * * * why *,1,* * * * why *,"(open-ended, 1)"
* * * * how *,1,* * * * how *,"(open-ended, 1)"
how * * have * *,1,how * * have * *,"(yes/no, 0)"
have * * * * *,1,have * * * * *,"(yes/no, 0)"


In [23]:
# question_trigger_words = question_trigger_words[question_trigger_words["count"] > 1]
question_trigger_words["prediction"] = question_trigger_words["type"].apply(lambda word: word[1])
question_trigger_words["prediction"].to_dict()

import json

data = question_trigger_words["prediction"].to_dict()

with open('question_data.json', 'w') as outfile:
    json.dump(data, outfile)

In [24]:
test_data = []

In [25]:
count = 0
success_count = 0

for i, questions in enumerate(categories):
    print("Category: "+str(i+1)+"/"+str(len(categories)))
    for index, question in questions.sample(n=100, random_state=0).iterrows():
        doc = nlp(question["question"])
        token = doc[0]
        if token.pos_ in ["AUX", "ADV"]:
            prediction, confidence = predict(doc)
            success = prediction == question["questionType"]
            test_data.append({"question": question["question"], "questionType": question["questionType"], "prediction": prediction, "success": success, "confidence": confidence})
            if success:
                success_count += 1
            count += 1
test_results = pd.DataFrame.from_dict(test_data)
test_results

print(success_count / count)

Category: 1/1


ValueError: Cannot take a larger sample than population when 'replace=False'

In [26]:
count = 0
success_count = 0

for i, questions in enumerate(categories):
    print("Category: "+str(i+1)+"/"+str(len(categories)))
    for index, question in questions.sample(n=1000, random_state=0).iterrows():
        print(question["question"])
        prediction = dict_predict(question["question"])
        if prediction > 0.5:
            category = "open-ended"
        elif prediction < 0.5:
            category = "yes/no"
        else:
            category = "unknown"
        print(category)
        success = category == question["questionType"]
        test_data.append({"question": question["question"], "questionType": question["questionType"], "prediction": prediction, "success": success, "confidence": confidence})
        if success:
            success_count += 1
        count += 1
test_results = pd.DataFrame.from_dict(test_data)
test_results

print(success_count / count)

Category: 1/1


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
avr_length_open

In [ ]:
avr_length_closed 

In [ ]:
test_results

In [ ]:
dict_predict("Does this come with a remote for each or does the same remote work both?")